# 03 - Entrainement du Modele de Diffusion (DDPM)

Ce notebook entraine un DDPM (Denoising Diffusion Probabilistic Model)
pour generer des images satellites synthetiques de secheresse.

**Pipeline :**
1. Preparer les images du domaine secheresse (generees par CycleGAN ou simulees)
2. Initialiser le DDPM (U-Net + scheduler)
3. Entrainer (~4-6h sur GPU T4 Colab)
4. Generer des echantillons
5. Evaluer la qualite visuelle

In [ ]:
# Setup (decommenter sur Colab)
# !git clone https://github.com/aymenssf/SatelliteGAN-Climate-Agriculture.git
# %cd SatelliteGAN-Climate-Agriculture
# !pip install -q -r requirements.txt

import sys
import os
sys.path.insert(0, os.path.join(os.getcwd()))

import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

from src.config import DEVICE, DIFFUSION, IMAGE_SIZE
from src.dataset import get_agricultural_dataset, split_dataset
from src.preprocessing import (
    get_eval_transform, simulate_drought, tensor_to_numpy
)
from src.diffusion.train import DiffusionTrainer
from src.evaluation.visualization import (
    show_image_grid, show_comparison, plot_training_losses
)

print(f"Device : {DEVICE}")
print(f"Configuration DDPM : {DIFFUSION}")

## 1. Preparation des donnees

On entraine le DDPM sur les images du domaine secheresse.

Deux options :
- **Option A** : Utiliser les images transformees par le CycleGAN (meilleur)
- **Option B** : Utiliser les images de secheresse simulee (plus simple)

On utilise l'option B par defaut. Pour l'option A, charger un checkpoint CycleGAN.

In [ ]:
from torchvision import transforms

# Charger les images et appliquer la simulation de secheresse
raw_transform = transforms.Resize(IMAGE_SIZE)
raw_dataset = get_agricultural_dataset(transform=raw_transform)
train_set, val_set, _ = split_dataset(raw_dataset)

# Creer le dataset de secheresse
eval_transform = get_eval_transform()

def prepare_drought_dataset(dataset, transform, n_max=None):
    """Prepare les images de secheresse normalisees."""
    images = []
    n = min(len(dataset), n_max) if n_max else len(dataset)

    for i in range(n):
        img, _ = dataset[i]
        if isinstance(img, torch.Tensor):
            img_pil = transforms.ToPILImage()(img)
        else:
            img_pil = img

        # Appliquer la secheresse
        drought_pil = simulate_drought(img_pil, severity=0.6)
        img_tensor = transform(drought_pil)
        images.append(img_tensor)

    return torch.stack(images)

print("Preparation des images de secheresse...")
train_drought = prepare_drought_dataset(train_set, eval_transform)
print(f"Dataset de secheresse : {train_drought.shape}")

In [ ]:
# Visualiser quelques echantillons du dataset d'entrainement
show_image_grid(train_drought[:16], n_cols=4,
                title='Echantillons du dataset secheresse (entrainement DDPM)')

In [ ]:
# DataLoader
train_dataset = TensorDataset(train_drought)
train_loader = DataLoader(
    train_dataset,
    batch_size=DIFFUSION['batch_size'],
    shuffle=True,
    drop_last=True
)

print(f"Nombre de batches par epoch : {len(train_loader)}")

## 2. Visualisation du processus de diffusion

Avant d'entrainer, visualisons comment le bruit est ajoute progressivement.

In [ ]:
from src.diffusion.scheduler import LinearNoiseScheduler

scheduler = LinearNoiseScheduler(
    n_timesteps=DIFFUSION['n_timesteps'],
    beta_start=DIFFUSION['beta_start'],
    beta_end=DIFFUSION['beta_end']
)

# Prendre une image
sample_img = train_drought[0:1]  # (1, 3, 64, 64)

# Afficher a differents timesteps
timesteps = [0, 50, 100, 250, 500, 750, 999]
fig, axes = plt.subplots(1, len(timesteps), figsize=(3 * len(timesteps), 3))

for i, t in enumerate(timesteps):
    t_tensor = torch.tensor([t])
    noisy, _ = scheduler.add_noise(sample_img, t_tensor)
    img_np = tensor_to_numpy(noisy[0])
    axes[i].imshow(img_np)
    axes[i].set_title(f't = {t}', fontsize=11)
    axes[i].axis('off')

plt.suptitle('Processus forward de diffusion (ajout de bruit)',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. Initialisation et entrainement

In [ ]:
# Creer le trainer
trainer = DiffusionTrainer()

# Compter les parametres
n_params = sum(p.numel() for p in trainer.model.parameters())
print(f"Parametres du U-Net : {n_params:,}")

In [ ]:
# Entrainer le modele
# Reduire n_epochs pour un test rapide (ex: 5-10 epochs)
# Pour l'entrainement complet, utiliser DIFFUSION['n_epochs'] (150)
N_EPOCHS = DIFFUSION['n_epochs']  # mettre 5 pour un test rapide

history = trainer.train(train_loader, n_epochs=N_EPOCHS)

## 4. Courbe de perte

In [ ]:
plot_training_losses(history, title='DDPM - Perte de debruitage (MSE)')

## 5. Generation d'images

In [ ]:
# Generer des images (sampling rapide)
print("Generation d'images (sampling rapide)...")
generated = trainer.generate(n_samples=16, use_ema=True, fast=True)

show_image_grid(
    generated.cpu(), n_cols=4,
    title='Images generees par le DDPM (secheresse synthetique)'
)

In [ ]:
# Comparaison avec les vraies images de secheresse
show_comparison(
    train_drought[:8].cpu(), generated[:8].cpu(),
    n_samples=4,
    labels=('Secheresse (dataset)', 'Secheresse (DDPM)'),
    title='Comparaison : images reelles vs generees'
)

## 6. Resume

**Observations :**
- Le DDPM apprend a generer des images satellites de secheresse
- Les images generees montrent des textures et couleurs coherentes
- Le sampling rapide (50 pas) produit des resultats raisonnables
- L'EMA ameliore sensiblement la qualite des generations

**Prochaine etape :** Evaluation quantitative avec SSIM, PSNR et FID.